In [119]:
# multivariate multi-step encoder-decoder lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Embedding
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping
# import tensorflow_hub as hub
import numpy as np
# import tensorflow_text
from sklearn.neighbors import NearestCentroid
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pandas as pd
import itertools
from random import sample 


from google.cloud import bigquery
from collections import defaultdict
import numpy as np
import logging
import random
from tqdm import tqdm


In [120]:
'''Experimental Parameters'''


LSTM_Units=100
Embedding_layer_size=20
sequence_length=5


In [137]:

def read_data(path):
    '''read sequences from '''
    with open(path, 'r') as f:
        seqs= f.read().split('\n')
    return seqs

def filter_sequence(seqs, min_length=3):
    '''filter a sequnce on minimum length'''
    filtered_seqs=[]
        
    for val in seqs:
        temp= [value for value in val.split(" ") if value != '']
        if len(temp)>= min_length:
            filtered_seqs.append(val)
    return filtered_seqs


def train_test_split(data, train_ratio=0.25):
    '''shuffle and split data into train test split'''
    random.shuffle(data)
    train_range= int(len(data)*train_ratio)
    return data[:train_range], data[train_range:]

def merge_seq_arr(arr):
    '''Merge array of items to space seperated sequence'''
    seq=""
    for item in arr: 
         seq+=item + " "
    return seq
    
def split_fixed_length_sequence(data, max_length=9):
    '''split sequences on fixed lengths to increse data points'''
    for seq in data: 
        if len(seq)> max_length:
            pass

        
def chunks(lst, n=8):
    """return successive n-sized chunks from lst."""
    slices=[]
    lst = lst.split(" ")
    for i in range(n, len(lst)+n, n):
        slices.append(merge_seq_arr(lst[i-n:i]))

    return slices

def extend_data_points(data, chunk_size=9):
    extended_data=[]
    for seqs in data:
        extended_data+=chunks(seqs, chunk_size)
    return extended_data



def create_vocab(base_seq):
    token_mapping={"<pad>":1.0}
    iter_val=2.0

    for seqs in base_seq:
        tokens=[value for value in seqs.split(" ") if value != '']
        unique_token=list(set(tokens))

        for token in unique_token:
            if token not in token_mapping:
                token_mapping[token] = iter_val
                iter_val+=1

    print(iter_val)

    import json 

    with open("vocab_.json", "w") as outfile:
        json.dump(token_mapping, outfile)
        
    return token_mapping 


def tokenizer(token_mapping, seqs, max_length=3 ):
    
    labels=[]
    input_ids=[]
    
    for seq in tqdm(seqs):
        encoded_sequence=[]
        mask_sequence=[]
        

        
        #breaking sequence in tokens
        seq=[value for value in seq.split(" ") if value != '']
        
        #appending encoded items to sequence
        for token in seq:
            encoded_sequence.append(token_mapping[token])
            
        

        labels.append(encoded_sequence.pop())
        
        
        #padding to max length
        if len(encoded_sequence)< max_length:
            for i in range(max_length-len(encoded_sequence)):
                encoded_sequence.append(token_mapping['<pad>'])
        

        input_ids.append(np.array(encoded_sequence))

    return labels, input_ids

In [138]:

#initialize model input data lists
input_ids =[]
mask =[]
labels= []
path = "seqs.txt"

    
base_seq= read_data(path)

token_mapping= create_vocab(base_seq)

fil_seq= filter_sequence(base_seq, min_length=sequence_length-1)
test_seq, train_seq= train_test_split(fil_seq)

updated_train= extend_data_points(train_seq, chunk_size=sequence_length+1)
updated_test= extend_data_points(test_seq, chunk_size=sequence_length+1)

updated_train= filter_sequence(updated_train)
updated_test= filter_sequence(updated_test)





759.0


In [139]:


labels_train, input_ids_train= tokenizer(token_mapping, updated_train, max_length=sequence_length)
labels_test, input_ids_test= tokenizer(token_mapping, updated_test, max_length=sequence_length)

vocab_size= len(token_mapping.keys()) +1


X_train, Y_train=np.array(input_ids_train), np.array(labels_train)
X_test, Y_test=np.array(input_ids_test), np.array(labels_test)


n_values = len(token_mapping) +1
labels_train= np.eye(n_values)[Y_train.astype(int)]
labels_test= np.eye(n_values)[Y_test.astype(int)]




100%|██████████| 22412/22412 [00:00<00:00, 571945.91it/s]


In [140]:

# n_steps_in, n_steps_out = 3, 1
# # covert into input/output
# X = np.load('data/X.data.npy' , allow_pickle=True)
# y = np.load('data/y.data.npy',allow_pickle=True)
# print(X.shape)
# # the dataset knows the number of features, e.g. 2
# n_features = 1

# embedding_vecor_length = 64
# vocab_size=len(y[0])
# # max_length=3

In [141]:
type(Y_test[0])

numpy.float64

In [ ]:

# define model
model = Sequential()
model.add(Embedding(vocab_size, Embedding_layer_size))

model.add(Bidirectional(LSTM(LSTM_Units, activation='relu', dropout=0.4, recurrent_dropout=0.2)))

model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=25, restore_best_weights=True)
model.fit(X_train, labels_train, batch_size=32, epochs=100, verbose=1,  validation_data=(X_test, labels_test), callbacks=[es])
# demonstrate prediction

model.save('data/model.h5')

Epoch 1/100
2365/2365 [==============================] - 24s 10ms/step - loss: 5.2877 - accuracy: 0.0203 - val_loss: 4.9153 - val_accuracy: 0.0340
Epoch 2/100
2365/2365 [==============================] - 22s 9ms/step - loss: 4.8298 - accuracy: 0.0322 - val_loss: 4.8224 - val_accuracy: 0.0371
Epoch 3/100
2365/2365 [==============================] - 24s 10ms/step - loss: 4.7317 - accuracy: 0.0368 - val_loss: 4.7784 - val_accuracy: 0.0403
Epoch 4/100
2365/2365 [==============================] - 22s 9ms/step - loss: 4.6649 - accuracy: 0.0417 - val_loss: 4.7432 - val_accuracy: 0.0453
Epoch 5/100
2365/2365 [==============================] - 22s 9ms/step - loss: 4.6119 - accuracy: 0.0468 - val_loss: 4.7186 - val_accuracy: 0.0481
Epoch 6/100
2365/2365 [==============================] - 23s 10ms/step - loss: 4.5676 - accuracy: 0.0514 - val_loss: 4.7162 - val_accuracy: 0.0498
Epoch 7/100
2365/2365 [==============================] - 23s 10ms/step - loss: 4.5261 - accuracy: 0.0561 - val_loss: 4.71